In [ ]:
import pandas as pd
import os
from google.colab import drive
from scipy import stats
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Psychologia/Magisterka/magisterka Psyche/Mateusz_Kamyczura_Praca_Magisterska/Kody/Baza danych'
Output_path = path + '/Output'

Mounted at /content/drive


In [ ]:
zasadyABC = pd.read_excel(Output_path + '/zasadyABC.xlsx')
zasadyXYZ = pd.read_excel(Output_path + '/zasadyXYZ.xlsx')

In [ ]:
ABC = pd.read_excel(Output_path + '/ABC.xlsx')
XYZ = pd.read_excel(Output_path + '/XYZ.xlsx')

In [ ]:
def test_proporcji_50plus(df, kolumna='ProcentZasad'):
    """
    Sprawdza, czy istotnie więcej niż połowa badanych odwoływała się do zasad w co najmniej 50% przypadków.

    Argumenty:
    df : pandas.DataFrame – tabela z kolumną procentów (np. 'ProcentZasad')
    kolumna : str – nazwa kolumny z procentem odwołań do zasad

    Zwraca:
    Wyniki testu Z
    """

    # Tworzenie zmiennej zero-jedynkowej
    df['czy_>=50proc'] = df[kolumna] >= 50

    liczba_tak = df['czy_>=50proc'].sum()
    liczba_nie = (~df['czy_>=50proc']).sum()
    liczba_wszystkich = len(df)

    print(f"📊 Liczba badanych z ≥50% zasad: {liczba_tak}/{liczba_wszystkich}")

    # Test Z dla jednej proporcji
    stat_z, p_z = proportions_ztest(count=liczba_tak, nobs=liczba_wszystkich, value=0.5, alternative='larger')

    print(f"\n🔹 Test proporcji Z:")
    print(f"Statystyka Z = {stat_z:.4f}, p = {p_z:.4f}")
    if p_z < 0.05:
        print("✅ Wniosek: Istotnie więcej niż połowa badanych odwoływała się do zasad ≥50% przypadków.")
    else:
        print("❌ Wniosek: Brak istotnych dowodów, że więcej niż połowa badanych używała zasad ≥50% przypadków.")



In [ ]:
test_proporcji_50plus(ABC)

📊 Liczba badanych z ≥50% zasad: 33/42

🔹 Test proporcji Z:
Statystyka Z = 4.5126, p = 0.0000
✅ Wniosek: Istotnie więcej niż połowa badanych odwoływała się do zasad ≥50% przypadków.


In [ ]:
test_proporcji_50plus(XYZ)

📊 Liczba badanych z ≥50% zasad: 28/41

🔹 Test proporcji Z:
Statystyka Z = 2.5171, p = 0.0059
✅ Wniosek: Istotnie więcej niż połowa badanych odwoływała się do zasad ≥50% przypadków.


In [ ]:
def procent_poprawnych_dla_grup_z_IZ(df):
    """
    Liczy procent poprawnych odpowiedzi osobno dla matematyków i niematematyków,
    z podziałem na użycie intuicji (I/Z == 1) i zasad (I/Z == 2),
    oraz dodatkowo sumarycznie dla wszystkich uczestników (mat = 's').

    Argumenty:
    df : pandas.DataFrame
        Musi zawierać kolumny 'CzyPoprawna', 'mat', 'I/Z'

    Zwraca:
    pandas.DataFrame z kolumnami:
        'mat', 'I/Z', 'ProcentPoprawnych'
    """

    # Filtrowanie tylko odpowiedzi udzielonych
    df_filtr = df[df['CzyPoprawna'] != 0].copy()

    # Poprawna = 1, Niepoprawna = 0 (zamiast -1)
    df_filtr['PoprawnaBinarna'] = (df_filtr['CzyPoprawna'] == 1).astype(int)

    # Grupowanie po 'mat' i 'I/Z'
    wynik_grupowy = df_filtr.groupby(['mat', 'I/Z'])["PoprawnaBinarna"].mean().reset_index()
    wynik_grupowy["ProcentPoprawnych"] = wynik_grupowy["PoprawnaBinarna"] * 100
    wynik_grupowy = wynik_grupowy.drop(columns="PoprawnaBinarna")

    # Sumaryczne wartości dla każdej kategorii I/Z (niezależnie od mat)
    suma = df_filtr.groupby("I/Z")["PoprawnaBinarna"].mean().reset_index()
    suma["mat"] = "s"
    suma["ProcentPoprawnych"] = suma["PoprawnaBinarna"] * 100
    suma = suma.drop(columns="PoprawnaBinarna")

    # Łączenie obu wyników
    wynik_koncowy = pd.concat([wynik_grupowy, suma], ignore_index=True)

    return wynik_koncowy


In [ ]:
procent_poprawnych_dla_grup_z_IZ(zasadyABC)

,mat,I/Z,ProcentPoprawnych
0,0,1,76.170213
1,0,2,88.650307
2,1,1,70.229008
3,1,2,90.000000
4,s,1,74.043716
5,s,2,89.417989


In [ ]:
procent_poprawnych_dla_grup_z_IZ(zasadyXYZ)

,mat,I/Z,ProcentPoprawnych
0,0,1,59.487179
1,0,2,69.816273
2,1,1,54.676259
3,1,2,71.118012
4,s,1,56.659619
5,s,2,70.412518


In [ ]:
def test_poprawnosci_zasady_vs_intuicja(df):
    """
    Sprawdza, czy odwołania do zasad (I/Z == 2) wiążą się z wyższym odsetkiem poprawnych odpowiedzi
    niż odwołania do intuicji (I/Z == 1).

    Argumenty:
    df : pandas.DataFrame – tabela z kolumnami "I/Z" (1: intuicja, 2: zasady) oraz "CzyPoprawna" (1: poprawna, -1: błędna, 0: brak odpowiedzi)
    """

    # Filtrujemy tylko odpowiedzi, gdzie ktoś odpowiedział
    df = df[df["CzyPoprawna"] != 0]

    # Grupujemy po strategii
    intuicja = df[df["I/Z"] == 1]
    zasady = df[df["I/Z"] == 2]

    # Liczymy liczbę poprawnych i wszystkich odpowiedzi
    poprawne_intuicja = (intuicja["CzyPoprawna"] == 1).sum()
    n_intuicja = len(intuicja)

    poprawne_zasady = (zasady["CzyPoprawna"] == 1).sum()
    n_zasady = len(zasady)

    print(f"📊 Intuicja: {poprawne_intuicja}/{n_intuicja} poprawnych")
    print(f"📊 Zasady:   {poprawne_zasady}/{n_zasady} poprawnych")

    # Test Z proporcji
    stat_z, p_z = proportions_ztest(count=[poprawne_zasady, poprawne_intuicja],
                                     nobs=[n_zasady, n_intuicja],
                                     alternative='larger')  # Sprawdzamy czy zasady > intuicja

    print(f"\n🔹 Test proporcji Z:")
    print(f"Statystyka Z = {stat_z:.4f}, p = {p_z:.4f}")
    if p_z < 0.05:
        print("✅ Wniosek: Odwołania do zasad wiążą się z istotnie lepszą poprawnością odpowiedzi.")
    else:
        print("❌ Wniosek: Brak istotnej różnicy w poprawności między zasadami a intuicją.")



In [ ]:
test_poprawnosci_zasady_vs_intuicja(zasadyABC)

📊 Intuicja: 271/366 poprawnych
📊 Zasady:   676/756 poprawnych

🔹 Test proporcji Z:
Statystyka Z = 6.6542, p = 0.0000
✅ Wniosek: Odwołania do zasad wiążą się z istotnie lepszą poprawnością odpowiedzi.


In [ ]:
test_poprawnosci_zasady_vs_intuicja(zasadyXYZ)

📊 Intuicja: 268/473 poprawnych
📊 Zasady:   495/703 poprawnych

🔹 Test proporcji Z:
Statystyka Z = 4.8447, p = 0.0000
✅ Wniosek: Odwołania do zasad wiążą się z istotnie lepszą poprawnością odpowiedzi.
